<a href="https://colab.research.google.com/github/openingsound/Hands-On_Machine_Learning/blob/master/Batch_Normalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Batch Normalization

## 개념
* NN의 입력 분포 변환는 일어나지 않는것이 좋다. 
  * NN 파라미터들이 그 분포에 새로 적응해야 하기 때문!
* 학습을 원활하게 하기 위해 효율적인 입력 분포는 일정하게 유지되는 입력 분포
* 네트워크를 여러 층을 쌓은 경우에 하위 네트워크에 대한 입력 분포의 효유성에도 동일
  * 앞단의 layer의 output의 분포도 일정하게 유지하는 것이 좋다. 

* 또 다른 문제점
  * sigmoid를 사용할 경우 절대값이 일정 수준이상(6이상) 으로 큰 입력에 대해서는 grad가 0 이된다. 
    * x의 분포가 0 으로부터 멀어진다면 Saturation이 됨(Saturated Regime)

## Internal Covariate Shift
* Covariate shift
  * train data와 test data의 data distribution이 다른 현상
  * x,y간의 대응을 학습 시킬 때 x에 대한 분포가 바뀌면 학습을 다시 해야 한다!

*  네트워크의 학습 도중에 파라미터의 변화로 인한 네트워크 Activation(출력)들의 분포 변화
* 가중치가 업데이트 될 때 마다 각 미니 배치 후 네트워크의 다음층에 대한 입력 분포가 변경 된다. 
* 학습 효율을 높이기위해서는 이를 줄여야 한다. 
  * 역전파 에서는 뒤에서 부터 오기 때문에 이전층의 입력 분포 변화는 고려하지 않는다. 즉 같은 분포로 입력이 들어 올 것이라고 가정하고 학습되는 것 이다. 
  * 10번층에 있는 층의 가중치가 1번 층처럼 앞쪽 층의 가중치의 변화에 영향을 덜 받는다. 
* 입력값들이 Whitening(평균이 0, Unit Variance,입력값이 서로 Decorrelated)된다면 NN이 훨씬 빠르게 수렴 할 것이다. 또한 모든 레이어들이 같은 Whitening방식을 공유한다면 훨씬 이득을 가질 수 있다. 


## Batch Normaliztoin의 목적과 효과
* 파라미터 업데이트 이후마다 트레이닝셋 전체의 분석이 필요하지 않을 뿐 아니라 미분도 가능해서 Backpropagation을 구하는것이 용이한 Normalization lr을 맘편히 증가 시키자!
* 좀더 엄밀히 하면 앞선 층의 매개변수가 바뀌었을 때 지금 층의 값이 받아 들여서 학습하게 될 값의 분포를 제한한다. (평균과 분산을 제한 함으로서)
  * 뒤쪽 층은 더 쉽게 학습할 수 있다. 입력 값의 분포가 바뀌더라도 조금만 바뀌는 것이다. 
  * 앞쪽 층의 매개변수와 뒤쪽 층의 매개변수 간의 관계를 약화 시킨다. 
    * 각 층이 다른 층과 상관 없이 스스로 학습한다.
* 규제 효과 
  * 미니 배치가 미니 배치의 평균과 분산에 따라 값을 조정 할 것이다. 
  * 미니 배치에서 계산한 평균과 분산은 전체 데이터로부터 계산한 것에 비해 다소 노이즈를 가지고 있다. (상대적으로 작은 데이터에서 추정한 것이기 때문)
    * 은닉층에 노이즈를 추가하는건 은닉층이 다른 은닉층에 너무 의존하지 않도록 해 준다. 
    * 단
      * 잡음이 아주 작다보니 일반화 효과가 그리 크지는 않다.  
      * 배치 정규화와 드롭아웃을 함께 사용 할 수 있다. (강력한 일반화)
    * 큰 배치 사이즈를 쓰면 일반화 효과가 줄어 든다. 
#### Caution
* 규제를 목적으로 사용하지는 말기!


## Navie approach: Whitening
이 논문의 목표는 internal covariate shift를 줄이는 것이다.

* input distribution을 zero mean, unit variance를 가지는 normal distribution으로 normalize 시키는 것으로 문제를 해결할 수 있고 이를 whitening이라고 한다. 
* 다중공선성이 생긴다.
* naive한 방법의 2가지 문제점
  * inverse의 square root를 계산해야 하기 때문에 필요한 계산량이 많다.
  * mean과 variance 세팅은 어떻게 할 것인가? 전체 데이터를 기준으로 mean variance를 training마다 계산하면 계산량이 많이 필요하다.
* 공분산 행렬을 구한다. (스케일을 고려하지 않은 선형적인 분포 성향)
  * 공분산
    * 스케일 포함
    * 선형 관계 
* 이걸로 나누면 스케일과 상관계수를 고려하지 않아도 된다.  

## 미니배치의 Statistics를 이용
* 위의 문제를 해결하기 위한 2가지 접근법
  * 각 차원들이 independent하다고 가정하고 각 차우너별로 따로 estimate하고 대신 표현형을 더 풍성하게 해 줄 linear transform도 학습한다.
  * 전체의 mean과 var대신 현재 batch 에서만 구한다음 inference할때만 real mean/var을 구한다. 
* 각각의 스칼라 Feature들을 독립적으로 정규화 하는 방식으로 진행된다. 
  * 각가의 Feature들의 mean 및var를 0과 1로 정규화 
  * $\gamma,  \beta$(std, mean)을 도입하여 모델이 학습이 되어갈때 같이 학습이 이뤄 지며 본래 모델의 Representation Power(복잡한 함수를 표현할 수 있는 힘)을 유지시키는 역할을 한다. 
* 미니배치 단위에서 정규화가 수행된다. 
  *  미니배치를 Stochastic하게 샘플링하여 정규화를 수행하게 된다. 
  * 각각의 미니배치는 각각의 Activation에 대하여 Mean 및 Variance를 추정하는데 사용된다. 
  * 이러한 방식을 통해서 정규화에 사용되는 Statistics가 Backpropagation에 활용될 수 있게 된다.




### Caution_
* 각 차원들의 Activation들을 독립적이라고 가정하고 각각의 Activation들 사이의 Joint Covariance를 고려하지 않는다
  * 단 이렇게 correlation을 무시하고 학습하는 경우 각가의 관계가 중요한 경우 제대로 되지 못한 학습을 하게 될 수도 있으므로 scaling과 shifting을 포함하는 $\gamma,  \beta$를 차원마다 학습 시킨다. 
* 파라미터 $\gamma,  \beta$로 Scale 및 Shift를 수행하여 출력값을 내보내게 된다. 이 때 $\gamma,  \beta$는 Backpropagation을 통해서 학습이 된다.  
* 은닉 유닛이 항상 평균 0, 분산 1을 갖는 것은 좋지많은 않다.
  * sigmoid 의 비 선형성을 살려야 한다. 
  * 은닉 유닛은 다양한 분포를 가져야 한다. 
  * $\tilde{z}^{l} = \gamma z^{l} + \beta $

### Batch Normalization의 특징

* 트레이닝을 위한 미니배치 단위에서 수행
* 미니배치 내의 한 Example 내에서의 Activation들은 각각 독립적이라고 가정
* 독립적인 각각의 Activation들은 정규화를 위하여 미니배치 내의 Example들의 Statistics를 이용
* $z^l = w^l a^{l-1}+ b^l $ 이지만 배치 정규화의 정규화 과정에서 z의 평균을 계산한 뒤에 빼주기 때문에 $b^l$들은 사라짐 즉 미니 배치의 모든 예시에 $b$를 더해줘도 아무런 영향을 끼치지 않는다. 
  * $z^l = w^l a^{l-1}$으로 하면 된다. 
  * $\tilde{z}^{l} = \gamma z^{l} + \beta $

## 레이어의 인퍼런스
*  트레이닝에서는 Activation의 정규화 과정에서 미니배치와의 Dependency를 고려하지만 인퍼런스에서도 이렇게 된다면 미니배치의 세팅에 따라서 결과가 달라지게 된다. 
   * 고정된 mean과 var필요
   * Sample mean과 Sample var를 이용하여 moving Avr  

## 활성화 함수 이전에 할지 뒤에 할지
* 논란이 많지만 보통 활성화 함수 이전에 많이 쓴다!

## CNN 에서의 BN
* Convolution layer 에서 activation fun에 입력되기 전에 $WX + b$로 weight가 적용 되었을 때 , $b$의 역할을 $\beta$가 대신 할 수 있기 때문에 $b$를 삭제한다.
* CNN의 경우 convolution의 성질을 유지시키고 싶기 때문에 각 채널을 기준으로 $\gamma, \beta$를 만든다. 
  * 최종적으로  $\gamma, \beta$는 각 channel에 대해 한개씩 총 , n개의 독립적인 Batch Normalization 변수 쌍이 생기게 된다. 

## Batch Renomalization
* 평균이나 분산 업데이트를 한번에 하는게 아니라 n번의 누적후 한번에 업데이트를 하자
* 평균과 분산에 노이즈 넣는걸 줄여 준다. 
* 더 평균적이다. 

## 충격적인 반전
  * ICS 를 줄이기 위해 개발을 시작하였지만 ICS에 의하여 좋아지는것은 아니라는 논문이 나왔다. 
  * https://arxiv.org/abs/1805.11604

In [ ]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

for fn in uploaded.keys(): # 업로드된 파일 정보 출력
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))


In [ ]:
from IPython.display import Image
Image('How Does BN help optimzation.png')

* ICS 감소 가 잘 되었는지 좌측 그래프를 보면 Standard와 Std+BN의 차이가 크지 않아 보인다.  BN이 ICS를 줄여주지 않는 것 처럼 보인다.
* 하지만 좌측의 그래프에서 주황과 파랑을 비교해 보면 BN을 쓰는게 성능적으로는 확실히 좋아 보인다. 
* BN을 쓰는 경우와 BN을 쓰고 거기에 Noisy를 넣은 경우를 보자 Noisy를 넣을 경우 BN만을 사용한 경우와 분산이 달라지므로 ICS가 커질 것이다. 하지만 좌측의 그래프를 보면 성능차이가 크게 나지 않는 것을 볼 수 있다. 

* 그럼 왜 BN은 성능을 향상 시키는 것일까?


* 위와 같은 분포의 변화가 아닌 BN을 거치면 무엇이 좋아지는지 보자.

In [ ]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

for fn in uploaded.keys(): # 업로드된 파일 정보 출력
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
from IPython.display import Image
Image('why BN is good optimzer 2.png')

* $W^t$에서 시작해서 학습을 해나간다고 하면 빨간 점선이 이상적인 이동이고 순간의 기울기가 a,b,c와 같이 나왔다면 기울기의 변화가 클 수록 손실이 상승하려고 하는것을 볼 수 있고 Loss가 크면 최적화가 어려워 진다.

In [ ]:
from google.colab import files
uploaded = files.upload() # 파일 업로드 기능 실행

for fn in uploaded.keys(): # 업로드된 파일 정보 출력
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

In [ ]:
from IPython.display import Image
Image('exp3.png')

* BN을 쓰면 기울기의 변화량이 크게 감소함을 관찰 할수 있었다. 이는 Loss의 변화량이 작다는 것을 의미하고 최적화를 잘하게 도와준다.
* BN을 쓰면 큰 기울기의 변화가 없어져 grad 가 부드럽고 안정적으로 변하게되어 Loss의 변화가 작아지고 더큰 학습률을 쓸수 있게 해 준다. 
* 이러한 효과라면 여러 규제들($l1, l2 , l_{inf}$)등이 있지만 BN의 성능이 더 좋다. 